<a href="https://colab.research.google.com/github/anushkathagle/Mask-Detection/blob/main/Mask_Detection_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip Mask_noMask.zip

In [4]:
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import LabelBinarizer, OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import os



In [5]:
import tensorflow as tf
tf.__version__


'2.17.1'

In [6]:
imagePath = []
for i in os.listdir("./dataset/"):
  for image in os.listdir("./dataset/" + i):
    path = "./dataset/" + i + "/" + image
    imagePath.append(path)

In [7]:
len(imagePath)

1376

In [8]:
data = []
labels = []

In [9]:
for i in imagePath:
  label = i.split("/")[-2]

  ### load image and resize it - 224, 224
  image = load_img(i, target_size=(224, 224))
  image = img_to_array(image)
  image = preprocess_input(image)

  data.append(image)
  labels.append(label)

data = np.array(data)
labels = np.array(labels)

In [10]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)


In [11]:
labels = to_categorical(labels)

In [12]:
trainX, testX, trainY, testY = train_test_split(data,
                                                labels,
                                                test_size = 0.20, random_state = 42)

In [13]:
trainY.shape

(1100, 2)

In [14]:
aug = ImageDataGenerator(rotation_range=25, zoom_range=0.1, width_shift_range=0.2, height_shift_range=0.2,
                         horizontal_flip=True, fill_mode = "nearest")

In [15]:


### Use Transfer Learning to train my Model with Mask DataSet
## We are using VGG19 pretrained Model

baseModel = VGG19(include_top=False, input_tensor=Input(shape=(224, 224, 3)))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [16]:

headModel = baseModel.output
headModel = Flatten(name = "flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

In [ ]:
# model.summary()

In [17]:

for layer in baseModel.layers:
  layer.trainable = False

In [18]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
H = model.fit(aug.flow(trainX, trainY, batch_size=32), validation_data=(testX, testY), epochs=5)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


35/35 ━━━━━━━━━━━━━━━━━━━━ 49s 906ms/step - accuracy: 0.8616 - loss: 2.6029 - val_accuracy: 0.9964 - val_loss: 0.1974
Epoch 2/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 51s 372ms/step - accuracy: 0.9813 - loss: 0.6899 - val_accuracy: 0.9964 - val_loss: 0.2445
Epoch 3/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 370ms/step - accuracy: 0.9834 - loss: 0.7455 - val_accuracy: 0.9819 - val_loss: 0.8241
Epoch 4/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 375ms/step - accuracy: 0.9820 - loss: 0.8974 - val_accuracy: 0.9891 - val_loss: 0.9953
Epoch 5/5
35/35 ━━━━━━━━━━━━━━━━━━━━ 15s 365ms/step - accuracy: 0.9827 - loss: 1.0685 - val_accuracy: 0.9928 - val_loss: 0.1355


In [ ]:
model.save("mask_noMask.h5")